In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

# Modèle du 2e exemple Pyomo:

maximiser $40 x_1 + 30 x_2$ (profit)

Sujet à $x_1 \leq 40$ (demand)

$x_1 + x_2 \leq 80$ (laborA)

$2x_1 + x_2 \leq 100$ (laborB)

$x_1, x_2 \geq 0$

# Option 1

In [ ]:
model = pe.ConcreteModel()

# variables
model.x1 = pe.Var(domain=pe.NonNegativeReals)  # ou within=
model.x2 = pe.Var(domain=pe.NonNegativeReals)

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=40 * model.x1 + 30 * model.x2)

# contraintes
model.demand = pe.Constraint(expr=model.x1 <= 40)
model.laborA = pe.Constraint(expr=model.x1 + model.x2 <= 80)
model.laborB = pe.Constraint(expr=2 * model.x1 + model.x2 <= 100)

# résolution
pe.SolverFactory('glpk').solve(model).write()

In [ ]:
model.pprint()

In [ ]:
model.display()

In [ ]:
model.x1.value  # idem pour x2

In [ ]:
pe.value(model.x1)

In [ ]:
model.profit.value

In [ ]:
pe.value(model.profit)

# Option 2
$x$ avec liste

In [ ]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=40 * model.x[1] + 30 * model.x[2])

# contraintes
model.demand = pe.Constraint(expr=model.x[1] <= 40)
model.laborA = pe.Constraint(expr=model.x[1] + model.x[2] <= 80)
model.laborB = pe.Constraint(expr=2 * model.x[1] + model.x[2] <= 100)

# résolution
pe.SolverFactory('glpk').solve(model).write()

In [ ]:
model.pprint()

# Option 3
objectif et contraintes avec dictionnaires

In [ ]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

profit = {1: 40, 
          2: 30}

demand = {1: 40}

labor_total = [80, 100]
labor = {1: [1, 2],
         2: [1, 1]}


# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=sum(profit[i] * model.x[i] for i in productList))

# contraintes
model.demand = pe.ConstraintList()
for i in demand:
    model.demand.add(expr=model.x[i] <= demand[i])

model.labor = pe.ConstraintList()
for j in range(len(labor_total)):
    model.labor.add(expr=sum(labor[i][j] * model.x[i] for i in productList) <= labor_total[j])

# résolution
pe.SolverFactory('glpk').solve(model).write()

In [ ]:
model.pprint()

# Option 4
utilisation de *rule*

In [ ]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

profit = {1: 40, 
          2: 30}

demand = {1: 40}

labor_total = [80, 100]
labor = {1: [1, 2],
         2: [1, 1]}


# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=sum(profit[i] * model.x[i] for i in productList))

# contraintes
model.demand = pe.ConstraintList()
for i in demand:
    model.demand.add(expr=model.x[i] <= demand[i])

def labor_rule(m, j):
    return sum(labor[i][j] * m.x[i] for i in productList) <= labor_total[j]
model.labor = pe.Constraint([0, 1], rule=labor_rule)    
# model.labor = pe.ConstraintList()
# for j in range(len(labor_total)):
#     model.labor.add(expr=sum(labor[i][j] * model.x[i] for i in productList) <= labor_total[j])

# résolution
pe.SolverFactory('glpk').solve(model).write()

In [ ]:
model.pprint()